In [2]:
#Importing essentials libraries
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense,Activation, Dropout
from keras.layers import LSTM

from sklearn import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,f1_score

from sklearn import tree

import collections
import datetime
import scipy

import graphviz
import plotly
from plotly.graph_objs import Scatter, Heatmap, Layout, Bar
plotly.offline.init_notebook_mode(connected=True)


Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscrimi

In [2]:
#Read dataset file
file = pd.read_csv("improved_spvm_2015-2018.csv")

#Manipulating the dataset to reflect the usage
file["DATE"] =  pd.to_datetime(file['JOUR'], format='%Y-%m-%d') #to transform the column into a datetime type

#transform the quart section to numeric (should be 0,1,2 for morning, afternoon and evening)
file = file.replace('jour',0)
file = file.replace('soir',1)
file = file.replace('nuit',2)

#split the 'DATE' columns into multiple ones.
file['JOUR'] = pd.DatetimeIndex(file['DATE']).day
file['MOIS'] = pd.DatetimeIndex(file['DATE']).month
file['ANNEE'] = pd.DatetimeIndex(file['DATE']).year

#Manipulating the 'CATEGORIE' column to transform it into a numeric
values_categories = np.unique(file['CATEGORIE'])
file = file.replace(values_categories, np.array([0, 1, 2, 3, 4 ,5])) 

#quick overview on the file to make sure it has been loaded properly
print(file.info())
#print(file.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93078 entries, 0 to 93077
Data columns (total 10 columns):
CATEGORIE    93078 non-null int64
JOUR         93078 non-null int32
QUART        93078 non-null int64
PDQ          93078 non-null int64
LAT          93078 non-null float64
LONG         93078 non-null float64
PLACE_ID     93078 non-null int64
DATE         93078 non-null datetime64[ns]
MOIS         93078 non-null int32
ANNEE        93078 non-null int32
dtypes: datetime64[ns](1), float64(2), int32(3), int64(4)
memory usage: 6.0 MB
None


In [3]:
#Firsts steps in the analysis : correlation analysis.
# In fact, we'll use several features for the SVM but before that, we need to make sure that each fetures
df = file

donnees = list(df.corr()) #getting the columns names.
vals = df.corr().values #getting the computed values for the correlation between each feature.

#print(df.corr())

plotly.offline.iplot({
    "data": [Heatmap(z=vals, y=donnees, x=donnees, colorscale='Viridis')],
    "layout": Layout(title='Correlation values')
})

NameError: name 'file' is not defined

In [1]:
#Building the dataset : 70% training and 30% test set.

scaler = MinMaxScaler(feature_range=(0,1))

datas = pd.DataFrame(file, columns=['JOUR', 'QUART', 'LAT', 'LONG', 'PLACE_ID', 'MOIS', 'ANNEE'])
values = preprocessing.scale(datas.values)
size = len(datas)
x = int(0.70*size)

trainset_x = values[0:x]
testset_x = values[x+1:-1]

values = file['CATEGORIE']
trainset_y = values[0:x]
testset_y = values[x+1:-1]

#print(trainset_x.shape)
#print(testset_x.shape)
#print(trainset_y.shape)
#print(testset_y.shape)


# Fit regression model
clf = tree.DecisionTreeRegressor()
clf = clf.fit(trainset_x, trainset_y)


NameError: name 'MinMaxScaler' is not defined

In [49]:
#Metrics evaluations

preds1 = clf.predict(testset_x)
ratio1 = np.logical_xor(testset_y, preds1)

preds1 = np.asarray(preds1,dtype=np.uint8)
preds1 = preds1.tolist()

print("Predictions = " + str(collections.Counter(preds1)))
print("     Values = " + str(collections.Counter(testset_y)))

coef_diff  = (preds1 != testset_y).sum()/float(len(preds1))
coef_simi  = (preds1 == testset_y).sum()/float(len(preds1))

f1_macro = f1_score(testset_y, preds1, average='macro')
f1_micro = f1_score(testset_y, preds1, average='micro')
f1_weighted = f1_score(testset_y, preds1, average='weighted')

data = [Bar(
            x=['F1 Score Macro', 'F1 Score Micro', 'F1 Score Weighted', 'Coef simi'],
            y=[f1_macro, f1_micro, f1_weighted, coef_simi],
    marker=dict(
        color=['rgba(204,204,204,1)', 'rgba(222,45,38,0.8)',
               'rgba(204,204,204,1)', 'rgba(204,204,204,1)',
               'rgba(204,204,204,1)'])
    )]

plotly.offline.iplot(data, filename='basic-bar')


Predictions = Counter({1: 8942, 3: 6847, 2: 6121, 4: 3913, 5: 2084, 0: 15})
     Values = Counter({1: 8737, 3: 7404, 2: 6141, 4: 4193, 5: 1426, 0: 21})


In [56]:
# Beautiful graphic

#dot_data = tree.export_graphviz(clf, out_file=None, feature_names=['JOUR', 'QUART', 'LAT', 'LONG', 'PLACE_ID', 'MOIS', 'ANNEE'], class_names=values_categories, filled=True, rounded=True, special_characters=True) 
#graph = graphviz.Source(dot_data)

dot_data = tree.export_graphviz(clf, out_file=None,feature_names=['JOUR', 'QUART', 'LAT', 'LONG', 'PLACE_ID', 'MOIS', 'ANNEE'], class_names=values_categories, filled=True, rounded=True, special_characters=True) 
graph = graphviz.Source(dot_data)
graph.render('PredCrimes')

KeyboardInterrupt: 